In [55]:
# 1) INTRODUCTION / BUSINESS PROBLEM

# I have just started my organic cosmetics company. I already have the prototypes for the cosmetics and a great website but am now looking for a good small commercial store in New York City from where I can start selling my physical products.

# Here the criterias I am looking for to find the best possible location for my organic cosmetics shop:

# - Rent price: the least expensive as possible
# - Location should be in a neighborhood with a high % of blacks or latinos, a high % of women and a low % of public assistance (from experience we sell our high-end skincare mostly to black and latina middleclass women)

# To find the best possible place we will combine Foursquare and publicly available data from nyc.gov and kaggle.com



# 2) DATA SECTION

# Price of Venues with the lowest possible rent => color coded map
# Available properties for rent/lease
# Zip codes with highest % of black and /or latin women + neighborhoods with lowest % of public assistance
# Cross available properties data with interesting zip codes for our business to get a shortlist of venues we want to visit and eventually lease


# 3) METHODOLOGY

# We will use several data sources from Foursquare to nyc.gov to shortlist available properties in preselected areas in NYC.
# We will first look at all venues in NYC and then narrow it down depending on our criterias.
# We will get data from nyc.gov to find the most interesting zip codes in NYC by focusing on our customer criterias.
# At the end we will cross available properties with areas where we want to do business. Goal is to shortlist properties we want to visit / lease for our new business.
 

In [56]:
# Import Libraries

import pandas as pd
import requests
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
mpl.style.use(['seaborn'])
!pip install geopy
from geopy.geocoders import Nominatim # module to convert address into latitude and longitude values
from IPython.display import Image
from IPython.core.display import HTML
from pandas.io.json import json_normalize # transform json files into pandas dataframe library
!pip install folium==0.5.0
import folium


# Foursquare API credentials and endpoint

CLIENT_ID = 'FIZ1RHKCAN24OPM2P4HX1PNB3VHRXB1JDWAXNPISVLATK4IE'
CLIENT_SECRET = 'AFJ5HIBZXPVNYPNKB03R5ULLPGWINFXG0CAMFA2NMR0ZNYGO'
VERSION = '20201228' # put today's date
URL = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, 5000)

In [57]:
# Define user agent to connect Foursquare location

address = '102 North End Ave, New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7149555 -74.0153365


In [58]:
url = 'https://api.foursquare.com/v2/venues/search?ll=40.7,-74&client_id=FIZ1RHKCAN24OPM2P4HX1PNB3VHRXB1JDWAXNPISVLATK4IE&client_secret=AFJ5HIBZXPVNYPNKB03R5ULLPGWINFXG0CAMFA2NMR0ZNYGO&v=20201228'
url

'https://api.foursquare.com/v2/venues/search?ll=40.7,-74&client_id=FIZ1RHKCAN24OPM2P4HX1PNB3VHRXB1JDWAXNPISVLATK4IE&client_secret=AFJ5HIBZXPVNYPNKB03R5ULLPGWINFXG0CAMFA2NMR0ZNYGO&v=20201228'

In [59]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6007689572f480192cc715af'},
 'response': {'venues': [{'id': '53ff2935498e161412b3e871',
    'name': 'Brooklyn Bridge Park - Pier 2 Yoga Court',
    'location': {'lat': 40.699657951708524,
     'lng': -73.99976911704667,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.699657951708524,
       'lng': -73.99976911704667}],
     'distance': 42,
     'cc': 'US',
     'city': 'Brooklyn',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['Brooklyn, NY', 'United States']},
    'categories': [{'id': '4bf58dd8d48988d102941735',
      'name': 'Yoga Studio',
      'pluralName': 'Yoga Studios',
      'shortName': 'Yoga Studio',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/gym_yogastudio_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1611098261',
    'hasPerk': False},
   {'id': '4ad35d08f964a520fae320e3',
    'name': 'NY Waterway Ferry - Wall St/Pier 11 Terminal',
   

In [60]:
# Keep relevant parts of venues

venues = results['response']['venues']

# Transform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.address,location.crossStreet,location.postalCode,venuePage.id
0,53ff2935498e161412b3e871,Brooklyn Bridge Park - Pier 2 Yoga Court,"[{'id': '4bf58dd8d48988d102941735', 'name': 'Y...",v-1611098261,False,40.699658,-73.999769,"[{'label': 'display', 'lat': 40.69965795170852...",42,US,Brooklyn,NY,United States,"[Brooklyn, NY, United States]",NaN,NaN,NaN,NaN
1,4ad35d08f964a520fae320e3,NY Waterway Ferry - Wall St/Pier 11 Terminal,"[{'id': '4bf58dd8d48988d12d951735', 'name': 'B...",v-1611098261,False,40.703253,-74.006095,"[{'label': 'display', 'lat': 40.70325299803113...",629,US,New York,NY,United States,"[South St (at Gouverneur Ln), New York, NY, Un...",South St,at Gouverneur Ln,NaN,NaN
2,51eabef6498e10cf3aea7942,Brooklyn Bridge Park - Pier 2,"[{'id': '4e74f6cabd41c4836eac4c31', 'name': 'P...",v-1611098261,False,40.699570,-73.997933,"[{'label': 'display', 'lat': 40.69957016220183...",180,US,Brooklyn,NY,United States,"[Furman St (Brooklyn Bridge Park Greenway), Br...",Furman St,Brooklyn Bridge Park Greenway,11201,NaN
3,3fd66200f964a520efe81ee3,The River Café,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",v-1611098261,False,40.703754,-73.994834,"[{'label': 'display', 'lat': 40.70375446317506...",603,US,Brooklyn,NY,United States,"[1 Water St (at Brooklyn Bridge), Brooklyn, NY...",1 Water St,at Brooklyn Bridge,11201,35406086
4,5284fe6811d2114d599b3e15,Brooklyn Bridge Park - Pier 3,"[{'id': '4e74f6cabd41c4836eac4c31', 'name': 'P...",v-1611098261,False,40.697803,-73.999443,"[{'label': 'display', 'lat': 40.69780332891386...",249,US,Brooklyn,NY,United States,"[Brooklyn Bridge Pk Greenway (at Furman St), B...",Brooklyn Bridge Pk Greenway,at Furman St,11201,NaN


In [61]:
# keep only columns with filled venue name, and anything related to location

filtered_columns = ['name' , 'categories'] + [col for col in dataframe.columns if col.startswith('location')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venues.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
dataframe_filtered

,name,categories,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,address,crossStreet,postalCode,id
0,Brooklyn Bridge Park - Pier 2 Yoga Court,Yoga Studio,40.699658,-73.999769,"[{'label': 'display', 'lat': 40.69965795170852...",42,US,Brooklyn,NY,United States,"[Brooklyn, NY, United States]",NaN,NaN,NaN,53ff2935498e161412b3e871
1,NY Waterway Ferry - Wall St/Pier 11 Terminal,Boat or Ferry,40.703253,-74.006095,"[{'label': 'display', 'lat': 40.70325299803113...",629,US,New York,NY,United States,"[South St (at Gouverneur Ln), New York, NY, Un...",South St,at Gouverneur Ln,NaN,4ad35d08f964a520fae320e3
2,Brooklyn Bridge Park - Pier 2,Pier,40.699570,-73.997933,"[{'label': 'display', 'lat': 40.69957016220183...",180,US,Brooklyn,NY,United States,"[Furman St (Brooklyn Bridge Park Greenway), Br...",Furman St,Brooklyn Bridge Park Greenway,11201,51eabef6498e10cf3aea7942
3,The River Café,American Restaurant,40.703754,-73.994834,"[{'label': 'display', 'lat': 40.70375446317506...",603,US,Brooklyn,NY,United States,"[1 Water St (at Brooklyn Bridge), Brooklyn, NY...",1 Water St,at Brooklyn Bridge,11201,3fd66200f964a520efe81ee3
4,Brooklyn Bridge Park - Pier 3,Pier,40.697803,-73.999443,"[{'label': 'display', 'lat': 40.69780332891386...",249,US,Brooklyn,NY,United States,"[Brooklyn Bridge Pk Greenway (at Furman St), B...",Brooklyn Bridge Pk Greenway,at Furman St,11201,5284fe6811d2114d599b3e15
5,Brooklyn Bridge Park Skating Rink,Roller Rink,40.699822,-73.998349,"[{'label': 'display', 'lat': 40.69982189729944...",140,US,Brooklyn,NY,United States,"[Pier 2, Brooklyn, NY, United States]",Pier 2,NaN,NaN,53af4e97498e0bd7ae45c926
6,Blue Marble Ice Cream,Ice Cream Shop,40.702837,-73.996274,"[{'label': 'display', 'lat': 40.70283693041026...",445,US,Brooklyn,NY,United States,"[Brooklyn Bridge Park - Pier 1, Brooklyn, NY 1...",Brooklyn Bridge Park - Pier 1,NaN,11201,4c3a5ad83849c92880e0c2b1
7,Pier 2 Handball Courts,Athletics & Sports,40.699557,-73.998061,"[{'label': 'display', 'lat': 40.699557, 'lng':...",170,US,New York,NY,United States,"[Brooklyn Bridge Park, New York, NY 11201, Uni...",Brooklyn Bridge Park,NaN,11201,5fb9462a7ef47d6dcf09b868
8,Boat Ramp @ Brooklyn Bridge Park,Scenic Lookout,40.701498,-73.997944,"[{'label': 'display', 'lat': 40.70149751178128...",240,US,Brooklyn,NY,United States,"[Pier 1 (Middagh & Cranberry St), Brooklyn, NY...",Pier 1,Middagh & Cranberry St,11201,4df50edee4cda09e6da88d90
9,Pier 3 Uplands,Park,40.697478,-73.998241,"[{'label': 'display', 'lat': 40.697478, 'lng':...",317,US,New York,NY,United States,"[New York, NY 11201, United States]",NaN,NaN,11201,5b4149016336be002c941465


In [62]:
# Generate map centred around Conrad Hotel
venues_map = folium.Map(location= [40.7149555,-74.0153365], zoom_start=12)

#add a red circle marker to represent the Conrad Hotel
folium.CircleMarker([latitude, longitude], radius=10, color='red', popup='Conrad Hotel', fill = True, fill_color = 'red', fill_opacity = 0.6).add_to(venues_map)

#add places from data on map
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker([lat, lng], radius=5,color='blue',popup=label, fill=True, fill_color='blue', fill_opacity=0.6).add_to(venues_map)

#display map
venues_map

In [63]:
# Given the limited amount of available data on Foursquare I checked out nyc.gov and downloaded a csv file with all NYC neighborhoods. I added the average rent price per square foot per borough that I found online.
# Source: https://www1.nyc.gov/site/planning/data-maps/open-data/bytes-archive.page?sorts[year]=0

# Let's import my csv file into Python first:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_4c24ee498856479a87bdda45b08f8b61 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='eIrP__5L1I3IK9Y45LhXadp-1QPjCWeD_Y4IC9b31wL9',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_4c24ee498856479a87bdda45b08f8b61.get_object(Bucket='organiccosmeticsstore-donotdelete-pr-yhcomnicj9r06z',Key='NYC_Real_Estate_5.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_8 = pd.read_csv(body)
df_data_8.head()

,address,city,boro,borocode,SQF Price,zipcode,latitude,longitude,facgroup,proptype,optype,opname,opabbrev,overlevel,facdomain
0,120 Schermerhorn Street,Brooklyn,Brooklyn,3,1047,11201,40.693682,-73.989693,"Offices, Training, and Testing",City Owned,Public,NYC Police Department,NYPD,NYPD: City,Administration of Government
1,132 West 125 Street,New York,Manhattan,1,1773,10027,40.811407,-73.953060,"Offices, Training, and Testing",City Leased,Public,NYC Human Resources Administration/Department ...,NYCHRA/DSS,NYCHRA/DSS: City,Administration of Government
2,3000 3 Avenue,Bronx,Bronx,2,317,10455,40.814710,-73.908593,"Offices, Training, and Testing",City Leased,Public,NYC Bronx Community Board 1,NYCCB201,NYCCB201: City,Administration of Government
3,52 Chambers Street,New York,Manhattan,1,1773,10007,40.713848,-74.007755,"Offices, Training, and Testing",City Owned,Public,NYC City Council,NYCCOUNCIL,NYCCOUNCIL: City,Administration of Government
4,90-15 Sutphin Boulevard,Jamaica,Queens,4,415,11435,40.701265,-73.809605,"Offices, Training, and Testing",City Leased,Public,NYC Office of the Mayor,NYCMO,NYCMO: City,Administration of Government


In [64]:
# Show data on a map:

map_nyc = folium.Map(location=[latitude, longitude], zoom_start=11)

# Adding markers to map
for lat, lng, address in zip(df_data_8['latitude'],  df_data_8['longitude'], df_data_8['address']):
 label = '{}'.format(address)
 label = folium.Popup(label, parse_html=True)
 folium.CircleMarker([lat, lng],radius=5,popup=label,color='blue',fill=True,fill_color='#3186cc',fill_opacity=0.7).add_to(map_nyc)
map_nyc

In [65]:
# Let's colorcode NYC locations depending on the average price per square meter. Green = affordable, orange = medium, red = expensive.

df_data_8['marker_color'] = pd.cut(df_data_8['SQF Price'], bins=4, labels=['green', 'yellow', 'orange', 'red'])

In [66]:
map_nyc = folium.Map(location=[40.7128, -74.0060],
            zoom_start=12)

for index, row in df_data_8.iterrows():
    folium.CircleMarker([row['latitude'], row['longitude']], radius=5, color=row['marker_color']).add_to(map_nyc)
       
map_nyc

In [67]:
# Red (=Manhattan) and Orange (=Brooklyn) are too expensive for our company, so we will drop these red and orange locations to focus on the green ones.
# Green = Bronx, Queens and Staten Island
# As we live in Queens, we will drop Staten Island because it is too far away and complicated to go there. We keep Queens and Bronx as possible locations

df_data_9 = df_data_8[df_data_8.boro != 'Manhattan']

In [68]:
df_data_10 = df_data_9[df_data_9.boro != 'Brooklyn']

In [69]:
df_data_11 = df_data_10[df_data_10.boro != 'Staten Island']

In [70]:
# Now we have our dataframe with only Queens and Bronx locations
df_data_11.head()

,address,city,boro,borocode,SQF Price,zipcode,latitude,longitude,facgroup,proptype,optype,opname,opabbrev,overlevel,facdomain,marker_color
2,3000 3 Avenue,Bronx,Bronx,2,317,10455,40.814710,-73.908593,"Offices, Training, and Testing",City Leased,Public,NYC Bronx Community Board 1,NYCCB201,NYCCB201: City,Administration of Government,green
4,90-15 Sutphin Boulevard,Jamaica,Queens,4,415,11435,40.701265,-73.809605,"Offices, Training, and Testing",City Leased,Public,NYC Office of the Mayor,NYCMO,NYCMO: City,Administration of Government,green
8,516 East Tremont Avenue,Bronx,Bronx,2,317,10457,40.847150,-73.898680,"Offices, Training, and Testing",City Leased,Public,NYC Police Department,NYPD,NYPD: City,Administration of Government,green
11,305 Rider Avenue,Bronx,Bronx,2,317,10451,40.820479,-73.925084,"Offices, Training, and Testing",City Leased,Public,NYC Human Resources Administration/Department ...,NYCHRA/DSS,NYCHRA/DSS: City,Administration of Government,green
12,2076 Washington Avenue,Bronx,Bronx,2,317,10457,40.847150,-73.898680,"Offices, Training, and Testing",City Owned,Public,NYC Business Integrity Commission,NYCBIC,NYCBIC: City,Administration of Government,green


In [71]:
# Show new map with only Bronx and Queens locations:

map_nyc = folium.Map(location=[latitude, longitude], zoom_start=11)

# Adding markers to map
for lat, lng, address in zip(df_data_11['latitude'],  df_data_11['longitude'], df_data_11['address']):
 label = '{}'.format(address)
 label = folium.Popup(label, parse_html=True)
 folium.CircleMarker([lat, lng],radius=5,popup=label,color='blue',fill=True,fill_color='#3186cc',fill_opacity=0.7).add_to(map_nyc)
map_nyc

In [72]:
# From personal research and experience we found out that people who spend the most on skincare and hair products in NYC are the BLACK and LATINO WOMEN.
# Therefore we want to find a location in a predominantly black or latino neighborhood in Queens or the Bronx. We also want to narrow down our search field by looking for black and latino communities that receive the least public assistance (in other words richer neighborhoods where people are more likely to buy high-end skincare)
# Source: csv file for race map: https://www.kaggle.com/new-york-city/ny-demographic-statistics-by-zip-code

body = client_4c24ee498856479a87bdda45b08f8b61.get_object(Bucket='organiccosmeticsstore-donotdelete-pr-yhcomnicj9r06z',Key='Demographics.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_12 = pd.read_csv(body)
df_data_12.head()

,JURISDICTION NAME,Boro,Address,COUNT PARTICIPANTS,COUNT FEMALE,PERCENT FEMALE,COUNT MALE,PERCENT MALE,COUNT GENDER UNKNOWN,PERCENT GENDER UNKNOWN,...,COUNT RECEIVES PUBLIC ASSISTANCE,PERCENT RECEIVES PUBLIC ASSISTANCE,COUNT NRECEIVES PUBLIC ASSISTANCE,PERCENT NRECEIVES PUBLIC ASSISTANCE,COUNT PUBLIC ASSISTANCE UNKNOWN,PERCENT PUBLIC ASSISTANCE UNKNOWN,COUNT PUBLIC ASSISTANCE TOTAL,PERCENT PUBLIC ASSISTANCE TOTAL,Unnamed: 48,Unnamed: 49
0,10001,Manhattan,128 West 31 Street,44,22,0.50,22,0.50,0,0,...,20,0.45,24,0.55,0,0,44,100,NaN,NaN
1,10002,Manhattan,117 Chrystie Street,35,19,0.54,16,0.46,0,0,...,2,0.06,33,0.94,0,0,35,100,NaN,NaN
2,10003,Manhattan,59 East 4 Street,1,1,1.00,0,0.00,0,0,...,0,0.00,1,1.00,0,0,1,100,NaN,NaN
3,10004,Manhattan,60 Broad Street,0,0,0.00,0,0.00,0,0,...,0,0.00,0,0.00,0,0,0,0,NaN,NaN
4,10005,Manhattan,120 Broadway,2,2,1.00,0,0.00,0,0,...,0,0.00,2,1.00,0,0,2,100,NaN,NaN


In [73]:
# Let's clean the dataset

# Only keep relevant columns:
df_data_13 = df_data_12[['JURISDICTION NAME','Boro','Address','PERCENT FEMALE','PERCENT HISPANIC LATINO','PERCENT BLACK NON HISPANIC','PERCENT RECEIVES PUBLIC ASSISTANCE']]

# Keep only the borough of Queens (we do Bronx afterwards):
df_data_14 = df_data_13[df_data_13.Boro == 'Queens']
df_data_14.head(100)

# We want to see neighborhoods for our business which have the following criterias:
# the highest % of black or latino residents (our main customers)
# the highest % of women (our main customers)
# the lowest % of public assistance received (we sell high-end products therefore need clients with a certain amount of money to be spent)

,JURISDICTION NAME,Boro,Address,PERCENT FEMALE,PERCENT HISPANIC LATINO,PERCENT BLACK NON HISPANIC,PERCENT RECEIVES PUBLIC ASSISTANCE
94,11101,Queens,30-30 Thomson Avenue,0.00,0.0,0.0,0.00
98,11105,Queens,45-02 Ditmars Boulevard,0.00,0.0,0.0,0.00
141,11354,Queens,32-02 Linden Place,0.50,0.5,0.0,0.00
142,11355,Queens,133-32 41 Road,0.00,0.0,0.0,0.00
148,11362,Queens,46-21 Little Neck Parkway,0.00,0.0,0.0,0.00
153,11368,Queens,59-17 Junction Boulevard,0.80,0.6,0.0,0.00
155,11370,Queens,75-20 Astoria Boulevard South,0.00,0.0,0.0,0.00
157,11372,Queens,82-11 37 Avenue,0.50,1.0,0.0,0.50
160,11375,Queens,118-35 Queens Boulevard,0.40,0.4,0.6,0.20
161,11377,Queens,24-55 Brooklyn Queens Ep W,0.00,0.0,0.0,0.00


In [74]:
# There are still too many Addresses in Queens. We do more data cleaning for better results. 

# We drop the rows with 0% women:
df_data_14 = df_data_14[df_data_14['PERCENT FEMALE'] != 0]

# Then we drop every row which is neither a latino, nor a black community ( = when the row has 0% in both categories):
df_data_15 = df_data_14 [~( df_data_14[['PERCENT HISPANIC LATINO','PERCENT BLACK NON HISPANIC']] == 0).all(axis=1)]

# Then we drop rows where % of received pubic assistance is higher than 40%: 
df_data_16 = df_data_15[df_data_15['PERCENT RECEIVES PUBLIC ASSISTANCE'] <= 0.40]
df_data_16

# Result = for QUEENS these 4 addresses  / zip codes are the best suited for our cosmetics business (lots of women, mostly black or latinas, neighborhood with very low received public assistance)

,JURISDICTION NAME,Boro,Address,PERCENT FEMALE,PERCENT HISPANIC LATINO,PERCENT BLACK NON HISPANIC,PERCENT RECEIVES PUBLIC ASSISTANCE
141,11354,Queens,32-02 Linden Place,0.5,0.5,0.0,0.0
153,11368,Queens,59-17 Junction Boulevard,0.8,0.6,0.0,0.0
160,11375,Queens,118-35 Queens Boulevard,0.4,0.4,0.6,0.2
186,11432,Queens,166-01 Hillside Avenue,1.0,0.0,0.5,0.0


In [75]:
# Now we do the same exercice for the borough BRONX:

# Only keep relevant columns:
df_data_13 = df_data_12[['JURISDICTION NAME','Boro','Address','PERCENT FEMALE','PERCENT HISPANIC LATINO','PERCENT BLACK NON HISPANIC','PERCENT RECEIVES PUBLIC ASSISTANCE']]

# Keep only the borough of Bronx:
df_data_14 = df_data_13[df_data_13.Boro == 'Bronx']
df_data_14.head(100)

# We drop the rows with 0% women:
df_data_20 = df_data_14[df_data_14['PERCENT FEMALE'] != 0]

# Then we drop every row which is neither a latino, nor a black community ( = when the row has 0% in both categories):
df_data_21 = df_data_20 [~( df_data_20[['PERCENT HISPANIC LATINO','PERCENT BLACK NON HISPANIC']] == 0).all(axis=1)]

# Then we drop rows where % of received pubic assistance is higher than 40%: 
df_data_22 = df_data_21[df_data_21['PERCENT RECEIVES PUBLIC ASSISTANCE'] <= 0.40]
df_data_22

# Result = For BRONX these 9 addresses / zip codes are the best suited for our cosmetics business (lots of women, mostly black or latinas, neighborhood with very low received public assistance)

,JURISDICTION NAME,Boro,Address,PERCENT FEMALE,PERCENT HISPANIC LATINO,PERCENT BLACK NON HISPANIC,PERCENT RECEIVES PUBLIC ASSISTANCE
58,10451,Bronx,305 Rider Avenue,0.51,0.54,0.37,0.29
59,10452,Bronx,1225 Gerard Avenue,0.71,0.59,0.29,0.18
62,10455,Bronx,3000 3 Avenue,0.63,0.19,0.74,0.26
63,10456,Bronx,1426 Boston Road,0.60,0.60,0.00,0.20
65,10458,Bronx,1 Fordham Plaza,0.48,0.46,0.44,0.37
73,10466,Bronx,4101 White Plains Road,0.60,0.22,0.77,0.26
74,10467,Bronx,3424 Kossuth Avenue,0.61,0.48,0.44,0.31
75,10468,Bronx,2501 Grand Concourse,0.57,0.43,0.44,0.28
77,10471,Bronx,5676 Riverdale Avenue,0.56,0.63,0.28,0.40


In [76]:
# Let's merge our 2 dataframes from Queens and Bronx together to display results on a map:
# New dataframe = df_data_16 (Queens results) + df_data_22 (Bronx results)

df_merged = pd.concat([df_data_16, df_data_22], axis=0)
df_merged

,JURISDICTION NAME,Boro,Address,PERCENT FEMALE,PERCENT HISPANIC LATINO,PERCENT BLACK NON HISPANIC,PERCENT RECEIVES PUBLIC ASSISTANCE
141,11354,Queens,32-02 Linden Place,0.50,0.50,0.00,0.00
153,11368,Queens,59-17 Junction Boulevard,0.80,0.60,0.00,0.00
160,11375,Queens,118-35 Queens Boulevard,0.40,0.40,0.60,0.20
186,11432,Queens,166-01 Hillside Avenue,1.00,0.00,0.50,0.00
58,10451,Bronx,305 Rider Avenue,0.51,0.54,0.37,0.29
59,10452,Bronx,1225 Gerard Avenue,0.71,0.59,0.29,0.18
62,10455,Bronx,3000 3 Avenue,0.63,0.19,0.74,0.26
63,10456,Bronx,1426 Boston Road,0.60,0.60,0.00,0.20
65,10458,Bronx,1 Fordham Plaza,0.48,0.46,0.44,0.37
73,10466,Bronx,4101 White Plains Road,0.60,0.22,0.77,0.26


In [77]:
# Now we can compare the dataframe we just created (all zip codes that would be great for our business) with the earlier dataset (available office space to rent in NYC)
# Find the best location for our cosmetics business

# df_merged = dataframe showing interesting zip codes for our business (high % of latin and/or black women live in these zip codes)
# df_data_11 = available office space to rent for our cosmetics business

# The link between both datasets is the zip code. If we find the same zip code in both files , we have a potential office space in an interesting neighborhood for us.

# we have a problem: the 2 datasets have 2 different column titles to define zip codes. I will rename the column "JURISDICTION NAME" in the dataset df_merged and call it "zipcode", so both dataframes match.

df_merged.rename(columns = {'JURISDICTION NAME':'zipcode'}, inplace = True)
df_merged

,zipcode,Boro,Address,PERCENT FEMALE,PERCENT HISPANIC LATINO,PERCENT BLACK NON HISPANIC,PERCENT RECEIVES PUBLIC ASSISTANCE
141,11354,Queens,32-02 Linden Place,0.50,0.50,0.00,0.00
153,11368,Queens,59-17 Junction Boulevard,0.80,0.60,0.00,0.00
160,11375,Queens,118-35 Queens Boulevard,0.40,0.40,0.60,0.20
186,11432,Queens,166-01 Hillside Avenue,1.00,0.00,0.50,0.00
58,10451,Bronx,305 Rider Avenue,0.51,0.54,0.37,0.29
59,10452,Bronx,1225 Gerard Avenue,0.71,0.59,0.29,0.18
62,10455,Bronx,3000 3 Avenue,0.63,0.19,0.74,0.26
63,10456,Bronx,1426 Boston Road,0.60,0.60,0.00,0.20
65,10458,Bronx,1 Fordham Plaza,0.48,0.46,0.44,0.37
73,10466,Bronx,4101 White Plains Road,0.60,0.22,0.77,0.26


In [78]:
# Now that both dataframes have the same column name to design zipcodes, let's see if we can find available office space (df_data_11) in the zipcodes we want (df_merged)

inner_join = pd.merge(df_merged, df_data_11, on ='zipcode', how ='inner') 
inner_join

,zipcode,Boro,Address,PERCENT FEMALE,PERCENT HISPANIC LATINO,PERCENT BLACK NON HISPANIC,PERCENT RECEIVES PUBLIC ASSISTANCE,address,city,boro,...,latitude,longitude,facgroup,proptype,optype,opname,opabbrev,overlevel,facdomain,marker_color
0,11354,Queens,32-02 Linden Place,0.50,0.50,0.00,0.00,32-02 Linden Place,Flushing,Queens,...,40.768208,-73.827403,"Offices, Training, and Testing",City Leased,Public,NYC Police Department,NYPD,NYPD: City,Administration of Government,green
1,11354,Queens,32-02 Linden Place,0.50,0.50,0.00,0.00,137-58 Northern Boulevard,Flushing,Queens,...,40.768208,-73.827403,"Offices, Training, and Testing",City Owned,Public,NYC Police Department,NYPD,NYPD: City,Administration of Government,green
2,11354,Queens,32-02 Linden Place,0.50,0.50,0.00,0.00,128 Street,Flushing,Queens,...,40.768208,-73.827403,"Offices, Training, and Testing",City Owned,Public,NYC Police Department,NYPD,NYPD: City,Administration of Government,green
3,11354,Queens,32-02 Linden Place,0.50,0.50,0.00,0.00,30-48 Linden Place,Flushing,Queens,...,40.768208,-73.827403,"Offices, Training, and Testing",City Owned,Public,NYC Department of Education,NYCDOE,NYCDOE: City,Administration of Government,green
4,11354,Queens,32-02 Linden Place,0.50,0.50,0.00,0.00,26-02 Ulmer Street,Flushing,Queens,...,40.768208,-73.827403,"Offices, Training, and Testing",City Owned,Public,NYC Police Department,NYPD,NYPD: City,Administration of Government,green
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,10466,Bronx,4101 White Plains Road,0.60,0.22,0.77,0.26,4101 White Plains Road,Bronx,Bronx,...,40.890964,-73.846239,"Offices, Training, and Testing",City Owned,Public,NYC Bronx Community Board 12,NYCCB212,NYCCB212: City,Administration of Government,green
68,10466,Bronx,4101 White Plains Road,0.60,0.22,0.77,0.26,4101 White Plains Road,Bronx,Bronx,...,40.890964,-73.846239,"Offices, Training, and Testing",City Owned,Public,NYC Department of Citywide Administrative Serv...,NYCDCAS,NYCDCAS: City,Administration of Government,green
69,10467,Bronx,3424 Kossuth Avenue,0.61,0.48,0.44,0.31,3424 Kossuth Avenue,Bronx,Bronx,...,40.869953,-73.865746,"Offices, Training, and Testing",City Owned,Public,NYC Human Resources Administration/Department ...,NYCHRA/DSS,NYCHRA/DSS: City,Administration of Government,green
70,10468,Bronx,2501 Grand Concourse,0.57,0.43,0.44,0.28,2501 Grand Concourse,Bronx,Bronx,...,40.868093,-73.899730,"Offices, Training, and Testing",City Leased,Public,NYC Administration for Childrens Services,NYCACS,NYCACS: City,Administration of Government,green


In [79]:
 # As we still have a lot of choices in Queens and Bronx we have decided to keep only locations in Queens which is where we live (=shorter commute to our business)

# Keep only the borough of Queens in our dataframe:

df_Queens = inner_join
df_Queens_2 = df_Queens[df_Queens.Boro == 'Queens']
df_Queens_2.head(5)

,zipcode,Boro,Address,PERCENT FEMALE,PERCENT HISPANIC LATINO,PERCENT BLACK NON HISPANIC,PERCENT RECEIVES PUBLIC ASSISTANCE,address,city,boro,...,latitude,longitude,facgroup,proptype,optype,opname,opabbrev,overlevel,facdomain,marker_color
0,11354,Queens,32-02 Linden Place,0.5,0.5,0.0,0.0,32-02 Linden Place,Flushing,Queens,...,40.768208,-73.827403,"Offices, Training, and Testing",City Leased,Public,NYC Police Department,NYPD,NYPD: City,Administration of Government,green
1,11354,Queens,32-02 Linden Place,0.5,0.5,0.0,0.0,137-58 Northern Boulevard,Flushing,Queens,...,40.768208,-73.827403,"Offices, Training, and Testing",City Owned,Public,NYC Police Department,NYPD,NYPD: City,Administration of Government,green
2,11354,Queens,32-02 Linden Place,0.5,0.5,0.0,0.0,128 Street,Flushing,Queens,...,40.768208,-73.827403,"Offices, Training, and Testing",City Owned,Public,NYC Police Department,NYPD,NYPD: City,Administration of Government,green
3,11354,Queens,32-02 Linden Place,0.5,0.5,0.0,0.0,30-48 Linden Place,Flushing,Queens,...,40.768208,-73.827403,"Offices, Training, and Testing",City Owned,Public,NYC Department of Education,NYCDOE,NYCDOE: City,Administration of Government,green
4,11354,Queens,32-02 Linden Place,0.5,0.5,0.0,0.0,26-02 Ulmer Street,Flushing,Queens,...,40.768208,-73.827403,"Offices, Training, and Testing",City Owned,Public,NYC Police Department,NYPD,NYPD: City,Administration of Government,green


In [80]:
# Show final data on a map:

Map_Queens = folium.Map(location=[latitude, longitude], zoom_start=11)

# Adding markers to map
for lat, lng, address in zip(df_Queens_2['latitude'],  df_Queens_2['longitude'], df_Queens_2['address']):
 label = '{}'.format(address)
 label = folium.Popup(label, parse_html=True)
 folium.CircleMarker([lat, lng],radius=5,popup=label,color='blue',fill=True,fill_color='#3186cc',fill_opacity=0.7).add_to(Map_Queens)
Map_Queens

In [81]:
# RESULT SECTION
# On the map above we have found the 4 locations which could be interesting for our cosmetics business and are available for lease.
# Thanks to the data we can now go visit these 4 places and then decide which one we want to lease.